<a href="https://colab.research.google.com/github/AyanRastogi/Agentic-AI-/blob/main/agenticai01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers gradio requests beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00


In [3]:
import re
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import gradio as gr

In [4]:
def clean_text(text):
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\[.*?\]|\(.*?\)', '', text)
    return text.strip()


In [5]:
def get_arxiv_paper(title):
    search_url = f"http://export.arxiv.org/api/query?search_query=all:{title}&start=0&max_results=1"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, "xml")
    entry = soup.find('entry')
    if not entry:
        return None, None
    paper_title = entry.title.text.strip()
    summary = entry.summary.text.strip()
    return paper_title, summary


In [6]:
summarizer = pipeline("summarization", model="google/flan-t5-large")
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu
Device set to use cpu


In [7]:
def generate_summary(paper_abstract):
    cleaned = clean_text(paper_abstract)
    summary = summarizer(cleaned, max_length=200, min_length=80, do_sample=False)[0]['summary_text']
    return summary

In [8]:
def answer_question(abstract, question):
    prompt = f"Context: {abstract}\n\nQuestion: {question}\nAnswer:"
    response = qa_pipeline(prompt, max_length=200)[0]['generated_text']
    return response.strip()

In [9]:
def paper_bot(title_input, question_input):
    paper_title, abstract = get_arxiv_paper(title_input)
    if not abstract:
        return " Paper not found. Try a different title."

    summary = generate_summary(abstract)
    answer_section = ""
    if question_input:
        answer = answer_question(abstract, question_input)
        answer_section = f"### Answer to your question:\n{answer}\n\n"

    return (
        f"##  Paper Title: {paper_title}\n\n"
        f"---\n\n"
        f"###  Summary:\n{summary}\n\n"
        f"{answer_section}"
        f"---\n"
        f"🔗 [Read full paper on arXiv](https://arxiv.org/search/?query={'+'.join(title_input.split())}&searchtype=all)"
    )


In [10]:
gr.Interface(
    fn=paper_bot,
    inputs=[
        gr.Textbox(label="Enter Research Paper Title"),
        gr.Textbox(label="Ask a Question About the Paper (optional)")
    ],
    outputs="markdown",
    title="📚 Research Paper Q&A Bot",
    description="Enter a paper title to get a summary and optionally ask a question based on the paper abstract."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93db626563379824ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
